Βήμα 1. Συλλογή δεδομένων:

Η συλλογή των δεδομένων έγινε από το kaggle στην διεύθυνση https://www.kaggle.com/datasets/jrobischon/wikipedia-movie-plots/data και η συλλογή περιέχει στοιχεία και υποθέσεις ταινιών.
Κατεβάζουμε τις απαραίτητες βιβλιοθήκες και περιορίζουμε το μέγεθος του dataset στις πρώτες 1000 εγγραφές λόγω περιορισμένων υπολογιστικών πόρων.

In [ ]:
import pandas as pd
import nltk
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
from collections import defaultdict
import json
import string
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import precision_score, recall_score, f1_score, average_precision_score
from rank_bm25 import BM25Okapi

nltk.download('stopwords')
nltk.download('punkt')

# Φόρτωση των δεδομένων
data = pd.read_csv('wiki_movie_plots_deduped.csv')
data = data.head(1000)

Βήμα 2. Προεπεξεργασία κειμένου (Text Processing):
Γίνεται διαγραφή των εγγραφών των οποίων είναι άδειο το plot, αφαιρούνται τα stopwords και εφαρμόζεται επεξεργασία stemming.

In [ ]:
stemmer = PorterStemmer()

# Διαγραφή των εγγραφων οι οποίες έχουν άδειο το κομμάτι plot
data = data.dropna(subset=['Plot'])

# Αποθήκευση του καθαρισμένου αρχείου
data.to_csv('cleaned_movies_dataset.csv', index=False)

# Καθορισμός των stopwords
stopwords = nltk.corpus.stopwords.words("english")

# Λειτουργία προετοιμασίας κειμένου με Stemming
def preprocess_text_with_stemming(text):
    # Αφαίρεση ειδικών χαρακτήρων
    text = ''.join([char for char in text if char not in string.punctuation])
    # Μετατροπή σε μικρά και tokenization
    text = text.lower()
    tokens = nltk.word_tokenize(text)
    # Αφαίρεση stopwords
    tokens = [token for token in tokens if token not in stopwords]
    # Stemming
    tokens = [stemmer.stem(token) for token in tokens]
    # Αφαίρεση πολλαπλών κενών
    text = " ".join(tokens)
    return text

data['Processed_Plot'] = data['Plot'].apply(preprocess_text_with_stemming)

# Αποθήκευση του καθαρισμένου dataset
data.to_csv('preprocessed_movies_dataset_with_stemming.csv', index=False)

Βήμα 3. Ευρετήριο (Indexing):
Χρησιμοποιούμε το inverted index για να συνδέσουμε τις λέξεις με τα έγγραφα στα οποία εμφανίζονται

In [ ]:
# Δημιουργία δομής ανεστραμμένου ευρετηρίου
inverted_index = defaultdict(list)

# Κατασκευή του ευρετηρίου
for idx, row in data.iterrows(): 
    doc_id = idx # Το ID του εγγράφου (μπορεί να είναι η γραμμή)
    words = row['Processed_Plot'].split() # Διάσπαση κειμένου σε λέξεις
    for word in words:
        if doc_id not in inverted_index[word]: # Αποφυγή διπλών εγγραφών
            inverted_index[word].append(doc_id)

# Εγγραφή του ευρετηρίου σε αρχείο
with open('inverted_index.json', 'w') as f:
    json.dump(inverted_index, f)
# Φόρτωση του ευρετηρίου σε αρχείο
with open('inverted_index.json', 'r') as f:
    inverted_index = json.load(f)

Βήμα 4. Μηχανή αναζήτησης (Search Engine)

α) Επεξεργασία ερωτήματος (Query Processing):
Η συνάρτηση boolean_search επεξεργάζεται τις λέξεις μέσω stemming,
το οποίο σημαίνει ότι κάθε λέξη μετατρέπεται στην ρίζα της (lemma) για να 
μειωθεί η πολυπλοκότητα της γλώσσας και να βελτιωθεί η αναζήτηση. Στη συνέχεια, 
αναγνωρίζει και εφαρμόζει λογικούς τελεστές Boolean όπως AND, OR και NOT για να εντοπίσει 
τα σχετικά έγγραφα στον αναστραμμένο ευρετήριο (inverted index). Το αποτέλεσμα είναι 
το σύνολο των εγγράφων που ικανοποιούν το ερώτημα αναζήτησης.

In [ ]:
def boolean_search(query, inverted_index):
    # Καθαρισμός και tokenization του ερωτήματος
    query = query.lower() # Μετατροπή σε μικρά γράμματα
    tokens = word_tokenize(query) # Tokenization
    tokens = [token for token in tokens if token not in stopwords]
    # Stemming σε κάθε λέξη
    stemmed_tokens = [stemmer.stem(token) for token in tokens]
    
    result_set = set()
    # Αναζήτηση για AND, OR, NOT
    if "and" in stemmed_tokens:
        terms = [term for term in stemmed_tokens if term != "and"]
        result_set = set(inverted_index.get(terms[0], []))
        for term in terms[1:]:
            result_set &= set(inverted_index.get(term, []))
    elif "or" in stemmed_tokens:
        terms = [term for term in stemmed_tokens if term != "or"]
        for term in terms:
            result_set |= set(inverted_index.get(term, []))
    elif "not" in stemmed_tokens:
        terms = [term for term in stemmed_tokens if term != "not"]
        result_set = set(inverted_index.keys()) - set(inverted_index.get(terms[0], []))
    else:
        result_set = set(inverted_index.get(stemmed_tokens[0], [])) # Απλή αναζήτηση

    return result_set

β) Κατάταξη αποτελεσμάτων (Ranking):

Βήμα 5. Αξιολόγηση συστήματος: